In [ ]:
#works 
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

sb.set()

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(2, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 2)
        )

    def forward(self, input):
        return self.main(input)

# Define the discriminator network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(2, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 2),
            nn.Linear(2, 1)
        )

    def forward(self, input):
        return self.main(input)

# Create instances of the generator and discriminator and move to device
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss function
criterion = nn.BCEWithLogitsLoss()

# Optimizers
gen_optimizer = optim.RMSprop(generator.parameters(), lr=0.001)
disc_optimizer = optim.RMSprop(discriminator.parameters(), lr=0.001)

# Function to generate noise
def sample_Z(m, n):
    return torch.Tensor(np.random.uniform(-1., 1., size=[m, n])).to(device)

# Load training data from m_hist.txt
def load_data(file_path):
    data = np.loadtxt(file_path, usecols=[0, 1])  # Load only the first two columns
    return torch.Tensor(data).to(device)

# Load your real training data
real_data_file = '/home/jovyan/blogs/basic-gans/code/m_hist.txt'  # Full path to your data file
x_plot = load_data(real_data_file)

# Training parameters
batch_size = min(256, len(x_plot))  # Adjust batch size to be smaller than or equal to the number of data points
nd_steps = 10
ng_steps = 10
num_iterations = 12001  # Change to 12001 for 12000 epochs

# Training loop
for i in range(num_iterations):
    for _ in range(nd_steps):
        # Sample a batch of real data
        indices = np.random.choice(len(x_plot), batch_size, replace=False)
        X_batch = x_plot[indices]
        Z_batch = sample_Z(batch_size, 2)

        # Real data labels are 1, generated data labels are 0
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Compute discriminator loss on real data
        real_outputs = discriminator(X_batch)
        real_loss = criterion(real_outputs, real_labels)

        # Compute discriminator loss on fake data
        fake_data = generator(Z_batch)
        fake_outputs = discriminator(fake_data.detach())
        fake_loss = criterion(fake_outputs, fake_labels)

        # Total discriminator loss
        disc_loss = real_loss + fake_loss

        # Backpropagation and optimization for discriminator
        disc_optimizer.zero_grad()
        disc_loss.backward()
        disc_optimizer.step()

    for _ in range(ng_steps):
        Z_batch = sample_Z(batch_size, 2)
        fake_data = generator(Z_batch)
        fake_outputs = discriminator(fake_data)

        # Generator loss
        gen_loss = criterion(fake_outputs, real_labels)

        # Backpropagation and optimization for generator
        gen_optimizer.zero_grad()
        gen_loss.backward()
        gen_optimizer.step()

    print(f"Iterations: {i}\t Discriminator loss: {disc_loss.item():.4f}\t Generator loss: {gen_loss.item():.4f}")

    if i % 1000 == 0:
        plt.figure()
        with torch.no_grad():
            g_plot = generator(sample_Z(batch_size, 2)).cpu().numpy()
        xax = plt.scatter(x_plot.cpu()[:, 0], x_plot.cpu()[:, 1], label='Real Data')
        gax = plt.scatter(g_plot[:, 0], g_plot[:, 1], label='Generated Data')

        plt.legend()
        plt.title(f'Samples at Iteration {i}')
        plt.tight_layout()
        plt.savefig(f'../plots/iterations/iteration_{i}.png')
        plt.close()

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import os

sb.set()

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(2, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 2)
        )

    def forward(self, input):
        return self.main(input)

# Define the discriminator network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(2, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 2),  # Assuming 2-dimensional output for feature representation
            nn.Linear(2, 1)
        )

    def forward(self, input):
        return self.main(input)

# Create instances of the generator and discriminator and move to device
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss function
criterion = nn.BCEWithLogitsLoss()

# Optimizers
gen_optimizer = optim.RMSprop(generator.parameters(), lr=0.001)
disc_optimizer = optim.RMSprop(discriminator.parameters(), lr=0.001)

# Function to generate noise
def sample_Z(m, n):
    return torch.Tensor(np.random.uniform(-1., 1., size=[m, n])).to(device)

# Load training data from m_hist.txt
def load_data(file_path):
    data = np.loadtxt(file_path, usecols=[0, 1])  # Load only the first two columns
    return torch.Tensor(data).to(device)

# Ensure directories exist for saving plots
plot_dirs = ['../plots/iterations', '../plots/features']
for plot_dir in plot_dirs:
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)

# Load your real training data
real_data_file = '/home/jovyan/blogs/basic-gans/code/m_hist.txt'  # Full path to your data file
x_plot = load_data(real_data_file)

# Training parameters
batch_size = min(256, len(x_plot))  # Adjust batch size to be smaller than or equal to the number of data points
nd_steps = 10
ng_steps = 10
num_iterations = 12001  # Change to 12001 for 12000 epochs

# Training loop
for i in range(num_iterations):
    for _ in range(nd_steps):
        # Sample a batch of real data
        indices = np.random.choice(len(x_plot), batch_size, replace=False)
        X_batch = x_plot[indices]
        Z_batch = sample_Z(batch_size, 2)

        # Real data labels are 1, generated data labels are 0
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Compute discriminator loss on real data
        real_outputs = discriminator(X_batch)
        real_loss = criterion(real_outputs, real_labels)

        # Compute discriminator loss on fake data
        fake_data = generator(Z_batch)
        fake_outputs = discriminator(fake_data.detach())
        fake_loss = criterion(fake_outputs, fake_labels)

        # Total discriminator loss
        disc_loss = real_loss + fake_loss

        # Backpropagation and optimization for discriminator
        disc_optimizer.zero_grad()
        disc_loss.backward()
        disc_optimizer.step()

    for _ in range(ng_steps):
        Z_batch = sample_Z(batch_size, 2)
        fake_data = generator(Z_batch)
        fake_outputs = discriminator(fake_data)

        # Generator loss
        gen_loss = criterion(fake_outputs, real_labels)

        # Backpropagation and optimization for generator
        gen_optimizer.zero_grad()
        gen_loss.backward()
        gen_optimizer.step()

    # Print and log the losses
    print(f"Iterations: {i}\t Discriminator loss: {disc_loss.item():.4f}\t Generator loss: {gen_loss.item():.4f}")

    # Save plots every 1000 iterations
    if i % 1000 == 0:
        # Plot generated and real data
        plt.figure()
        with torch.no_grad():
            g_plot = generator(sample_Z(batch_size, 2)).cpu().numpy()
        plt.scatter(x_plot.cpu()[:, 0], x_plot.cpu()[:, 1], label='Real Data')
        plt.scatter(g_plot[:, 0], g_plot[:, 1], label='Generated Data')
        plt.legend()
        plt.title(f'Samples at Iteration {i}')
        plt.tight_layout()
        plt.savefig(f'../plots/iterations/iteration_{i}.png')
        plt.close()

        # Plot feature transformations
        with torch.no_grad():
            real_features = discriminator(X_batch)
            fake_features = discriminator(fake_data)
            
            # Check if feature output has 2 dimensions for plotting
            if real_features.size(1) == 2:
                real_features = real_features.cpu().numpy()
                fake_features = fake_features.cpu().numpy()
                
                plt.figure()
                plt.scatter(real_features[:, 0], real_features[:, 1], alpha=0.5, label='Real Features')
                plt.scatter(fake_features[:, 0], fake_features[:, 1], alpha=0.5, label='Fake Features')
                plt.legend()
                plt.title(f'Features at Iteration {i}')
                plt.tight_layout()
                plt.savefig(f'../plots/features/feature_transform_{i}.png')
                plt.close()
            else:
                print(f"Feature output dimension mismatch at iteration {i}. Skipping feature plot.")


Iterations: 0	 Discriminator loss: 0.7887	 Generator loss: 0.6484
Feature output dimension mismatch at iteration 0. Skipping feature plot.
Iterations: 1	 Discriminator loss: 0.6986	 Generator loss: 0.6553
Iterations: 2	 Discriminator loss: 0.6785	 Generator loss: 0.6567
Iterations: 3	 Discriminator loss: 0.6588	 Generator loss: 0.7229
Iterations: 4	 Discriminator loss: 0.6013	 Generator loss: 0.8093
Iterations: 5	 Discriminator loss: 0.5340	 Generator loss: 0.9335
Iterations: 6	 Discriminator loss: 0.4551	 Generator loss: 1.1191
Iterations: 7	 Discriminator loss: 0.3804	 Generator loss: 1.2857
Iterations: 8	 Discriminator loss: 0.3381	 Generator loss: 1.1150
Iterations: 9	 Discriminator loss: 0.4068	 Generator loss: 1.1723
Iterations: 10	 Discriminator loss: 0.4599	 Generator loss: 1.0804
Iterations: 11	 Discriminator loss: 0.5186	 Generator loss: 1.1132
Iterations: 12	 Discriminator loss: 0.5606	 Generator loss: 1.1505
Iterations: 13	 Discriminator loss: 0.5844	 Generator loss: 1.0694